In [1]:
import pandas as pd
import numpy as np

In [2]:
#Find all possible orders of changing parameters, cf. p 178
import itertools
l_permutation = list(itertools.product([1, 2],[1,2],[1, 2],[1,2],[1, 2],[1,2],[1, 2],[1,2],[1, 2]))
df_permutation = pd.DataFrame(l_permutation,columns = ['beta','mu','p','delta','alpha','g_L','g_Z','g_Q','N_bar'])

In [3]:
class par: None
class moms: None
# Estimated parameters from calibration.     
estimates = pd.DataFrame(index=["beta","mu","p","delta","alpha","g_L","g_Z","g_Q","N_bar"])
estimates.index.name = "Name"
estimates["1984 - 2000"] = [0.953469,1.146507,0.029529,0.030853,0.357382,0.002629,0.016778,0.006778,0.488959]
estimates["2001 - 2016"] = [0.968215,1.172300,0.074540,0.039073,0.300462,0.006818,0.007149,0.004109,0.514776]

In [54]:
#Assumptions and settings
par.sigma = 0.5
par.theta = 12
par.b = -np.log(1-0.15)
par.bh = -np.log((2-np.exp(-par.b)))

#Define equations as functions of parameters only
def growthpop(par,moms):
    return par.g_L

def priceinvt(par,moms):
    return -par.g_Q

def ik(par,moms):
    return (1+par.g_Q)*(1+par.g_T)-(1-par.delta)

def EmpPop(par,moms):
    return par.N_bar

def ProfitY(par,moms):
    return (par.mu+par.alpha-1)/(par.mu)

def TFPgrowth(par,moms):
    moms.ProfitY = (par.mu+par.alpha-1)/(par.mu)
    par.g_T = (1+par.g_L)*(1+par.g_Z)**(1/(1-par.alpha))*(1+par.g_Q)**(par.alpha/(1-par.alpha)) - 1
    return par.g_T-(1-moms.ProfitY)*par.g_L-moms.ProfitY*(par.g_T+par.g_Q)

def ProfitK(par,moms):
    par.g_T = (1+par.g_L)*(1+par.g_Z)**(1/(1-par.alpha))*(1+par.g_Q)**(par.alpha/(1-par.alpha)) - 1
    par.g_PC = (1+par.g_T)/(1+par.g_L) - 1
    MOM = ((1-2*par.p)+par.p*np.exp(-par.bh*(1-par.theta)) + par.p*np.exp(-par.b*(1-par.theta)))**((1-par.sigma)/(1-par.theta)) 
    beta_star = par.beta * (1+par.g_PC)**(-par.sigma)*MOM
    r_star = 1/beta_star - 1
    return ((par.mu + par.alpha -1)/(par.alpha))*(r_star + par.delta + par.g_Q*(1+r_star))

def RF(par,moms):
    par.g_T = (1+par.g_L)*(1+par.g_Z)**(1/(1-par.alpha))*(1+par.g_Q)**(par.alpha/(1-par.alpha)) - 1
    par.g_PC = (1+par.g_T)/(1+par.g_L) - 1
    MOM = ((1-2*par.p)+par.p*np.exp(-par.bh*(1-par.theta)) + par.p*np.exp(-par.b*(1-par.theta)))**((1-par.sigma)/(1-par.theta)) 
    beta_star = par.beta * (1+par.g_PC)**(-par.sigma) *MOM
    MOM2 = ((1-2*par.p)+par.p*np.exp(-par.bh*(1-par.theta)) + par.p*np.exp(-par.b*(1-par.theta)))
    MOM3 = ((1-2*par.p)+par.p*np.exp(-par.bh*(-par.theta)) + par.p*np.exp(-par.b*(-par.theta))) 
    return MOM2/(MOM3*beta_star)-1


#The ISSUE...
def PD(par,moms):
    
    MOM = ((1-2*par.p)+par.p*np.exp(-par.bh*(1-par.theta)) + par.p*np.exp(-par.b*(1-par.theta)))**((1-par.sigma)/(1-par.theta))     
    beta_star = par.beta * (1+par.g_PC)**(-par.sigma) *MOM
    
    
    par.b = -np.log(1-0.15)
    par.bh = -np.log((2-np.exp(-par.b)))
    par.g_T = (1+par.g_L)*(1+par.g_Z)**(1/(1-par.alpha))*(1+par.g_Q)**(par.alpha/(1-par.alpha)) - 1
    par.g_PC = (1+par.g_T)/(1+par.g_L) - 1
    MOM = ((1-2*par.p)+par.p*np.exp(-par.bh*(1-par.theta)) + par.p*np.exp(-par.b*(1-par.theta)))**((1-par.sigma)/(1-par.theta))     
    
    bs1 = 0.9344697870097948
    bs2 = 0.9506301883608526
    p1 = 0.029529
    p2 = 0.074540
    
    if par.beta == estimates.loc["beta","1984 - 2000"]:
        beta_star = bs1/MOM
        bs = beta_star*MOM
    elif par.beta == estimates.loc["beta","2001 - 2016"]:
        beta_star = bs2/MOM
        bs = beta_star*MOM
    
    
    r_star = 1/bs - 1
    return ((1+par.g_T)/(r_star-par.g_T))
    #return ((bs)*(1+par.g_T)/(1-(bs)*(1+par.g_T)))
    


In [55]:
#Vary all parameters one by one and store values. 
result = [(ProfitK(par,moms),ProfitY(par,moms),RF(par,moms), growthpop(par,moms), priceinvt(par,moms),PD(par,moms), TFPgrowth(par,moms), ik(par,moms), EmpPop(par,moms))
            for par.beta in [estimates.loc["beta","1984 - 2000"],estimates.loc["beta","2001 - 2016"]]
            for par.mu in [estimates.loc["mu","1984 - 2000"],estimates.loc["mu","2001 - 2016"]]
            for par.p in [estimates.loc["p","1984 - 2000"],estimates.loc["p","2001 - 2016"]]
            for par.delta in [estimates.loc["delta","1984 - 2000"],estimates.loc["delta","2001 - 2016"]]
            for par.alpha in [estimates.loc["alpha","1984 - 2000"],estimates.loc["alpha","2001 - 2016"]]
            for par.g_L in [estimates.loc["g_L","1984 - 2000"],estimates.loc["g_L","2001 - 2016"]]
            for par.g_Z in [estimates.loc["g_Z","1984 - 2000"],estimates.loc["g_Z","2001 - 2016"]]
            for par.g_Q in [estimates.loc["g_Q","1984 - 2000"],estimates.loc["g_Q","2001 - 2016"]]
            for par.N_bar in [estimates.loc["N_bar","1984 - 2000"],estimates.loc["N_bar","2001 - 2016"]]
         ]
df = pd.DataFrame(result, columns =["ProfitK","ProfitY","RF", "growthpop","priceinvt","PD","TFPgrowth","ik","EmpPop"])

In [56]:
#Join with df of all possible orders from above
df = df_permutation.join(df)

In [57]:
moments = ["ProfitK","ProfitY","RF","PD","ik","TFPgrowth","priceinvt","growthpop","EmpPop"]
parameters = ["beta","mu","p","delta","alpha","g_L","g_Z","g_Q","N_bar"]

df_results = pd.DataFrame(columns=parameters)
df_results["moment"] = moments
df_results = df_results.set_index("moment",drop=True)

#Take means of all possible orders, conditional on one parameter. 
for mom in moments:
    for parm in parameters:
        result = df.loc[(df[parm] == 2),mom].mean()-df.loc[(df[parm] == 1),mom].mean() 
        df_results.loc[mom,parm] = result    

#Print and format results.
df_results_formatted = df_results.copy()        
df_results_formatted = df_results_formatted.astype(float) # Convert all to floats (they appear to be strings?)
#df_results_formatted['sum'] = df_results_formatted.sum(axis=1, skipna=True)    # Sum rows  
#df_results_formatted['p2'] = [moms.ProfitK,moms.s_K,moms.RF,moms.PD,moms.ik,moms.TFPgrowth,moms.g_Q,moms.g_L,moms.N_bar]
#df_results_formatted['p1'] = df_results_formatted['p2']-df_results_formatted['change']
#df_results_formatted = df_results_formatted[["p1","p2","change","beta","mu","p","delta","alpha","g_L","g_Z","g_Q","N_bar"]] # Change order – sum first

scalar = [1 if i=="PD" else 100 for i in df_results_formatted.index.tolist()]  #Create list, 1 for PD, 100 for all other
df_results_formatted = df_results_formatted.multiply(scalar,axis=0)

for var in df_results_formatted.columns.tolist()[:]:
    df_results_formatted[var] = df_results_formatted[var].map('${:,.2f}$'.format)

### Our result of decomposition

In [58]:
df_results_formatted

beta       mu        p   delta    alpha     g_L       g_Z  \
moment                                                                    
ProfitK    $-2.44$   $0.80$   $1.12$  $1.22$   $0.68$  $0.00$   $-1.13$   
ProfitY     $0.00$   $1.29$   $0.00$  $0.00$  $-4.91$  $0.00$    $0.00$   
RF         $-1.56$   $0.00$  $-2.34$  $0.00$  $-0.11$  $0.00$   $-0.72$   
PD         $18.93$   $0.00$   $0.00$  $0.00$  $-3.28$  $5.43$  $-16.39$   
ik          $0.00$   $0.00$   $0.00$  $0.82$  $-0.23$  $0.43$   $-1.46$   
TFPgrowth   $0.00$  $-0.03$   $0.00$  $0.00$  $-0.00$  $0.00$   $-0.84$   
priceinvt   $0.00$   $0.00$   $0.00$  $0.00$   $0.00$  $0.00$    $0.00$   
growthpop   $0.00$   $0.00$   $0.00$  $0.00$   $0.00$  $0.42$    $0.00$   
EmpPop      $0.00$   $0.00$   $0.00$  $0.00$   $0.00$  $0.00$    $0.00$   

               g_Q   N_bar  
moment                      
ProfitK    $-0.52$  $0.00$  
ProfitY     $0.00$  $0.00$  
RF         $-0.07$  $0.00$  
PD         $-1.76$  $0.00$  
ik         $-0.41$  $0.00$  
TFPgrowth   $0.04$  $0.00$  
priceinvt   $0.27$  $0.00$  
growthpop   $0.00$  $0.00$  
EmpPop      $0.00$  $2.58$

### Result using Farhi & Gourios MatLab-file

In [ ]:
from IPython.display import Image
Image("https://www.norgaardpetersen.dk/assets/PD.png")